In [2]:
!pip install hyperopt

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     ------ --------------------------------- 0.3/1.6 MB 7.9 MB/s eta 0:00:01
     ------------- -------------------------- 0.6/1.6 MB 7.0 MB/s eta 0:00:01
     -------------------- ------------------- 0.8/1.6 MB 7.4 MB/s eta 0:00:01
     --------------------------- ------------ 1.1/1.6 MB 7.0 MB/s eta 0:00:01
     ----------------------------------- ---- 1.4/1.6 MB 6.5 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.6 MB 6.7 MB/s eta 0:00:01
     ---------------------------------------  1.6/1.6 MB 6.7 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/200.5 kB ? eta -:--:--
     -------------------------------------- - 194.6/200.5 kB ? eta -:--:--
     -------------------------------------- - 194.6/200.5 kB ? eta -:--:--
    

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from hyperopt import fmin, tpe, hp
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [4]:
# 1. Reading data from CSV
def read_csv(file_path):
    return pd.read_csv(file_path)

# 2. Creating features
def create_features(data):
    # No feature creation for this example
    return data

# 3. Training a classifier model
def train_classifier(data):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return model, accuracy

# 4. Hyperparameter tuning with Hyperopt
def objective(params):
    model = RandomForestClassifier(**params)
    score = cross_val_score(model, X, y, cv=5).mean()
    return -score  # Minimize negative accuracy

# 5. Evaluating the model on the test set
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [5]:
file_path = ".././Sample_ML_Project/iris.csv"
data = read_csv(file_path)

In [6]:
data

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [7]:
# if __name__ == "__main__":
# Load data
file_path = ".././Sample_ML_Project/iris.csv"
data = read_csv(file_path)

# Create features
data = create_features(data)

# Split data into features and target
X = data.drop('Species', axis=1)
y = data['Species']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), X.columns)
        ],
        remainder='passthrough'
    )),
    ('classifier', RandomForestClassifier())
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy on test set: {accuracy}")

#Brute force appraoch try with 2 for loops

# Hyperparameter tuning using Tree of Parzen Estimators (TPE)---hyperpt is better 
space = {
    'n_estimators': hp.choice('n_estimators', range(10, 101)),
    'max_depth': hp.choice('max_depth', range(1, 21))
}

best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)

# Update the pipeline with the best hyperparameters


# Train the model with the best hyperparameters


# Evaluate the updated model


Model accuracy on test set: 1.0
100%|████████████████████████████████████████████████████████████| 100/100 [00:57<00:00,  1.72trial/s, best loss: -1.0]


In [8]:
best_params

{'max_depth': 17, 'n_estimators': 57}

In [9]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [10]:
# Update the pipeline with the best parameters
updated_pipeline = pipeline.set_params(**{'classifier__'+k:v for k,v in best_params.items()})

In [11]:
updated_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object'))])),
                ('classifier',
                 RandomForestClassifier(max_depth=17, n_estimators=57))])

In [12]:
# Train the model with the best hyperparameters
updated_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object'))])),
                ('classifier',
                 RandomForestClassifier(max_depth=17, n_estimators=57))])

In [14]:
# Evaluate the updated model
y_pred = updated_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy on test set: {accuracy}")

Model accuracy on test set: 1.0
